In [ ]:
# Half-Life as implemented by Duolingo 

In [61]:
import math
import numpy as np
from collections import defaultdict, namedtuple
from sys import intern
import pandas as pd
import os

In [62]:
current_dir = os.getcwd()

filename = 'df_processed.csv'
filepath = os.path.normpath(os.path.join(current_dir, '../data/processed/', filename))

chunk_size = 10000
chunks = []

for chunk in pd.read_csv(filepath, chunksize=chunk_size):
    chunk.drop_duplicates(inplace=True)
    chunk.dropna(inplace=True)
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)
df_users = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../data/features/', 'users_behaviur.csv')))
df_words = pd.read_csv(os.path.normpath(os.path.join(current_dir, '../data/features/', 'word_complexity_features.csv')), sep='\t')

df_1 = df.merge(df_words, on = 'lexeme_id', how='inner')
df_1['lang_combination'] = df_1['ui_language']+ '-' + df_1['learning_language']
df_2 = df_1.merge(df_users, on = ['user_id', 'lang_combination'], how='inner')
dff = df_2.drop(columns=['timestamp', 'lexeme_id', 'word', 'user_id', 'POS', 'person', 'number', 'gender', 'tense', 'def', 'session_seen', 'session_correct'])

In [63]:
a = pd.DataFrame({'lang':[1,2,3], 'halo':[0.01, 0, 0.99]})

In [64]:
dff.columns

Index(['p_recall', 'delta', 'learning_language_x', 'ui_language_x',
       'history_seen', 'history_correct', 'h_recall', 'word_len', 'SUBTLEX',
       'lang_combination', 'learning_language_y', 'ui_language_y', 'avg_delta',
       'avg_user_p_recall', 'std_delta', 'avg_h_recall'],
      dtype='object')

In [65]:
# Changes to dataset before fitting 
dff['p_recall'] = pclip(dff['p_recall'])
dff['delta'] = dff['delta']/(60*60*24) # convert time delta to days
# dff['half_life'] = hclip(-dff['delta']/np.log2(dff['p_recall']))
# tag_counts = dff['tags_list'].value_counts()
# rare_threshold = 1000
# dff['tags_list'] = dff['tags_list'].apply(lambda x: x if tag_counts[x] > rare_threshold else 'rare')


dff_final = dff.drop(columns=['learning_language_y', 'ui_language_y', 'learning_language_x', 'ui_language_x', 'avg_user_p_recall'], errors='ignore')
dff_final.dropna(inplace=True)

# original                 
# h = hclip(-t/(math.log(p, 2)))
# lang = '%s->%s' % (row['ui_language'], row['learning_language'])
# lexeme_id = row['lexeme_id']
# lexeme_string = row['lexeme_string']
# timestamp = int(row['timestamp'])
# user_id = row['user_id']
# seen = int(row['history_seen'])
# right = int(row['history_correct'])
# wrong = seen - right
# right_this = int(row['session_correct'])
# wrong_this = int(row['session_seen']) - right_this

# they left only lang combination 
# used lexeme id 
# used timestamp and lexeme_string and user_id 
# h = hclip(-t/(math.log(p, 2)))

In [66]:
dff_final

,p_recall,delta,history_seen,history_correct,h_recall,word_len,SUBTLEX,lang_combination,avg_delta,std_delta,avg_h_recall
0,0.999900,5.143600,3,3,1.000000,4,111241.0,en-es,2.475405e+06,2.879771e+06,0.954897
1,0.999900,0.069016,8,6,0.750000,5,3391.0,en-de,3.104417e+03,2.977079e+03,0.890225
2,0.750000,0.069016,6,5,0.833333,3,2484854.0,en-de,3.104417e+03,2.977079e+03,0.890225
3,0.888889,0.069016,6,5,0.833333,4,222707.0,en-de,3.104417e+03,2.977079e+03,0.890225
4,0.800000,0.069016,8,6,0.750000,4,143725.0,en-de,3.104417e+03,2.977079e+03,0.890225
...,...,...,...,...,...,...,...,...,...,...,...
12509996,0.800000,0.004259,6,4,0.666667,3,22761659.0,it-en,5.391088e+04,6.550006e+04,0.892502
12509997,0.800000,0.004259,4,4,1.000000,3,197654.0,it-en,5.391088e+04,6.550006e+04,0.892502
12509998,0.999900,0.004259,4,4,1.000000,5,24828.0,it-en,5.391088e+04,6.550006e+04,0.892502
12509999,0.600000,0.004259,3,2,0.666667,5,182580.0,it-en,5.391088e+04,6.550006e+04,0.892502


In [76]:
# datanstance object
instances = list()
Instance = namedtuple('Instance', 'p_recall delta fv half_life'.split())

# feature vector is a list of (feature, value) tuples
fv = []
fv.append((intern('history_seen'), np.sqrt(1+dff_final['history_seen'])))
fv.append((intern('history_correct'), np.sqrt(1+dff_final['history_correct'])))
fv.append((intern('h_recall'), dff_final['h_recall']))
fv.append((intern('word_len'), dff_final['word_len']))
fv.append((intern('SUBTLEX'), dff_final['SUBTLEX']))
fv.append((intern('lang_comb'), dff_final['lang_combination']))
fv.append((intern('avg_delta'), dff_final['avg_delta']))
fv.append((intern('SUBTLEX'), dff_final['SUBTLEX']))
fv.append((intern('std_delta'), dff_final['std_delta']))
fv.append((intern('avg_h_recall'), dff_final['avg_h_recall']))
# fv.append((intern('tags_list'), tags_list))

instances.append(Instance(p, t, fv, h))

NameError: name 'p' is not defined

In [ ]:
from collections import namedtuple
import numpy as np
import pandas as pd

# Define the Instance namedtuple
Instance = namedtuple('Instance', 'p_recall delta fv half_life')

# Function to transform a DataFrame row into an Instance object
def create_instances_from_dataframe(df):
    instances = []
    for _, row in df.iterrows():
        # Build the feature vector for this row
        fv = []
        fv.append((intern('history_seen'), np.sqrt(1 + row['history_seen'])))
        fv.append((intern('history_correct'), np.sqrt(1 + row['history_correct'])))
        fv.append((intern('h_recall'), row['h_recall']))
        fv.append((intern('word_len'), row['word_len']))
        fv.append((intern('SUBTLEX'), row['SUBTLEX']))
        fv.append((intern('lang_comb'), row['lang_combination']))
        fv.append((intern('avg_delta'), row['avg_delta']))
        fv.append((intern('SUBTLEX'), row['SUBTLEX']))
        fv.append((intern('std_delta'), row['std_delta']))
        fv.append((intern('avg_h_recall'), row['avg_h_recall']))
        # You can add other features as necessary

        # Create the Instance object
        instance = Instance(
            p_recall=row['p_recall'],
            delta=row['delta'],
            fv=fv,
            half_life=row['half_life']
        )

        # Append to the list of instances
        instances.append(instance)
    
    return instances


In [5]:
# Constants
min_half_life = 15.0 / (24 * 60)  # 15 minutes in days
max_half_life = 274.0            # 9 months
LN2 = math.log(2)

# Utility functions
def pclip(p):
    """Clip recall probability to avoid numerical issues."""
    return np.clip(p, 0.001, 0.9999)

def hclip(h):
    """Clip half-life to a reasonable range."""
    return np.clip(h, min_half_life, max_half_life)

def mae(l1, l2):
    # mean average error
    return mean([abs(l1[i] - l2[i]) for i in range(len(l1))])

def mean(lst):
    # the average of a list
    return float(sum(lst))/len(lst)

In [ ]:
# Define the HLR Model
class HalfLifeRegression:
    def __init__(self, learning_rate=0.001, hlwt=0.01, l2wt=0.1, sigma=1.):
        self.weights = defaultdict(float)  # Feature weights
        self.fcounts = defaultdict(int)    # Feature counts for adaptive learning rates
        self.learning_rate = learning_rate # Base learning rate
        self.hlwt = hlwt                   # Weight for half-life loss
        self.l2wt = l2wt                   # L2 regularization weight
        self.sigma = sigma
        if initial_weights is not None:
            self.weights.update(initial_weights)

    def halflife(self, inst):
        """Compute predicted half-life based on feature vector."""
        try:
            dp = sum([self.weights[k]*x_k for (k, x_k) in inst.fv]) # where inst.fv  is the feature vector
            return hclip(2 ** dp)
        except: 
            return MAX_HALF_LIFE
            

    def predict(self, inst):
        """Predict recall probability and half-life."""
        h_pred = self.halflife(inst)
        p_pred = 2 ** (-inst.delta / h) 
        return pclip(p_pred), h_pred

                
     def train_update(self, inst):
        """Update weights using one training instance."""
        p_pred, h_pred = self.predict(inst)

        # Compute gradients
        dlp_dw = 2*(p_pred - inst.p_recall)*(LN2 ** 2)*p_pred*(inst.delta/h_pred)
        dlh_dw = 2*(h_pred - inst.half_life)*LN2*h_pred

        # Update weights
        for (k, x_k) in inst.fv:
            rate = (1./(1+inst.p_recall)) * self.lrate / math.sqrt(1 + self.fcounts[k])
            self.weights[k] -= rate * dlp_dw * x_k  # Update for recall probability loss
            self.weights[k] -= rate * self.hlwt * dlh_dw * x_k  # Update for half-life loss
            self.weights[k] -= rate * self.l2wt * self.weights[k] / self.sigma**2  # L2 regularization
            self.fcounts[k] += 1


    def train(self, trainset):
        random.shuffle(trainset)
        for instance in trainset:
            self.train_update(inst)

    def evaluate(self, testset):
        """Evaluate the model on a test dataset."""
        results = {'p': [], 'h': [], 'pp': [], 'hh': [], 'slp': [], 'slh': []}
        for instance in testset:
            slp, slh, p, h = self.losses(inst)
            results['p'].append(inst.p_recall)     # ground truth
            results['h'].append(inst.h_recall)
            results['pp'].append(p_pred)         # predictions
            results['hh'].append(h_pred)
            results['slp'].append(slp)      # loss function values
            results['slh'].append(slh)
            mae_p = mae(results['p'], results['pp'])
            mae_h = mae(results['h'], results['hh'])
            total_slp = sum(results['slp'])
            total_slh = sum(results['slh'])
            total_l2 = sum([x**2 for x in self.weights.values()])
            total_loss = total_slp + self.hlwt*total_slh + self.l2wt*total_l2
        print(f"SLP Loss: {total_slp}, SLH Loss: {total_slh}, MAE_P: {mae_p}, MAE_H: {mae_h}, total loss {total_loss}")

In [ ]:
model = HalfLifeRegression()
model.train(trainset)
model.evaluate(testset)